In [31]:
import pandas as pd
import gpxpy
import requests

from datetime import datetime
from wordcloud import WordCloud
import matplotlib.pyplot as plt

from concurrent.futures import ThreadPoolExecutor

In [32]:
gpx_file = open('../data/SG200Miles2024.gpx', 'r')
gpx = gpxpy.parse(gpx_file)

In [33]:
data = []

for track in gpx.tracks:
    for segment in track.segments:
        for point in segment.points:
            data.append({'latitude': point.latitude, 'longitude': point.longitude})

df = pd.DataFrame(data)
df.head(3)

,latitude,longitude
0,1.34228,103.83519
1,1.34265,103.83574
2,1.34276,103.83592


In [34]:
def get_address(row):
    url = f'http://localhost:8080/reverse?format=json&lat={row["latitude"]}&lon={row["longitude"]}'
    response = requests.get(url)
    result = response.json()
    
    street_name = result["address"]["road"] if "road" in result["address"] else "Unknown"
    
    return f"{street_name}"

In [35]:
with ThreadPoolExecutor() as executor:
    addresses = list(executor.map(get_address, [row for _, row in df.iterrows()]))
df['address'] = addresses

df.head(3)

,latitude,longitude,address
0,1.34228,103.83519,MacRitchie Nature Trail
1,1.34265,103.83574,MacRitchie Nature Trail
2,1.34276,103.83592,MacRitchie Nature Trail


In [40]:
df = df[df['address'] != 'Unknown']
address_counts = df['address'].value_counts()

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(address_counts)

timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
filename = f"wordcloud_{timestamp}.png"
wordcloud.to_file(f"../images/{filename}")

plt.figure(figsize=(100, 50))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

TypeError: WordCloud.to_file() got an unexpected keyword argument 'dpi'